In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import pandas as pd
import os

In [ ]:
category = {}
data = []

with os.scandir("./data") as files:
    for file in files:
        # Only process .npy files
        if file.name.endswith('.npy'):
            try:
                category[file.path[25:-4]] = category.get(file.path[25:-4], len(category))

                # Load the numpy array with allow_pickle=True
                np_data = np.load(file.path, allow_pickle=True)
                img_num=0
                for i in np_data:
                    #only put the first 40000 arrays since there are too many
                    if img_num==40000: break
                    img_num+=1
                    #dividing by 255 to normalise the pixels
                    data.append({"Image": i / 255, "Name": category[file.path[25:-4]]})

            except EOFError:
                print(f"EOFError: Could not read {file.path}, skipping...")
            except Exception as e:
                print(f"Error loading {file.path}: {e}, skipping...")


#Organising the data in a dataframe using the pandas library
df = pd.DataFrame(data, columns=["Image", "Name"]).sample(frac=1).reset_index(drop=True)

names = reversed_dict = {value: key for key, value in category.items()}


In [ ]:
# Prepare training data
x_train = np.array([np.reshape(img, (784,)) for img in df["Image"]])
y_train = tf.keras.utils.to_categorical(df["Name"], num_classes=len(category))

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(300, input_shape=(784,), activation="relu"),
    tf.keras.layers.Dense(60, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(40, activation="relu"),
    tf.keras.layers.Dense(len(category), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


In [11]:
#saving the model
model.save("./model.keras")